In [314]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

 # device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device: ', device)

df = pd.read_csv('./output/colors_2024-03-22T18-04-05.csv')


Using device:  cpu


In [315]:
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    1080 non-null   object
 1   r       1080 non-null   int64 
 2   g       1080 non-null   int64 
 3   b       1080 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 33.9+ KB


,word,r,g,b
0,A,113,221,120
1,B,53,191,116
2,C,181,61,238
3,D,84,179,60
4,E,22,64,85
...,...,...,...,...
1075,castle,181,61,238
1076,broad,53,191,116
1077,long,113,58,208
1078,could,181,61,238


In [316]:
# turn loaded dataframe into torch tensors
from word_encoding import WordEncoder

we = WordEncoder()

inputs_list = we.encode_list(df['word'])
inputs = torch.tensor((len(inputs_list),1,45), dtype=torch.float)
inputs = torch.cat(inputs_list, dim=0)
print(inputs.shape)
torch.save(inputs, 'inputs0.pt')
print(inputs[0].shape)


torch.Size([1080, 45])
torch.Size([45])


In [317]:
outputs = torch.tensor(df.drop(['word'],axis='columns').values)
print(outputs.shape)
print(outputs[0].shape)
torch.save(outputs, 'outputs0.pt')

torch.Size([1080, 3])
torch.Size([3])


In [318]:
# create dataset class to take inputs & outputs
class Data(Dataset):
    # Constructor
    def __init__(self, inputs, outputs):
        self.x = inputs
        self.x = self.x.to(device)
        self.y = outputs
        self.y = self.y.to(device)
        self.len = len(inputs)
    
    # Getter
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    # Get number of samples
    def __len__(self):
        return self.len

In [319]:
# Making Model
class ColorPredictor(torch.nn.Module):
    #Constructor
    def __init__(self):
        super(ColorPredictor, self).__init__()
        self.linear1 = torch.nn.Linear(45, 20, device= device) #length of encoded word vectors
        self.linear2 = torch.nn.Linear(20,10, device= device)
        self.linear3 = torch.nn.Linear(10,3, device=device)
        self.reLU = torch.nn.ReLU()
        
    # Prediction
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.linear1(x)
        out = self.reLU(out)
        out = self.linear2(out)
        out = self.reLU(out)
        out = self.linear3(out)
        return out

In [320]:
# Create model

model = ColorPredictor()
model.to(device)


ColorPredictor(
  (linear1): Linear(in_features=45, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (reLU): ReLU()
)

In [321]:
# create train/test split
train_size = int(0.8 * len(inputs))
print(train_size)
print(len(inputs)- train_size)
train_data = Data(inputs[:train_size], outputs[:train_size])
test_data = Data(inputs[train_size:], outputs[train_size:])

864
216


In [322]:
# create DataLoaders
train_loader = DataLoader(dataset = train_data, batch_size=2)
test_loader = DataLoader(dataset = test_data, batch_size=2)

In [323]:
# criterion & optimizer
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
adam = torch.optim.Adam(model.parameters())
# adamw = torch.optim.AdamW(model.parameters())

In [326]:
# train & test loops
import copy
import numpy as np
torch.manual_seed(42)

epochs = 20

epoch_count = []
loss_values = []
test_loss_values = []
best_mse = 155.8818
best_weights = None


for epoch in range(epochs):
    # Train
    model.train()
    for x,y in train_loader:
        y_pred = model(x)
        loss = criterion(y_pred.float(), y.float())
        adam.zero_grad()
        loss.backward()
        adam.step()
        
        
     # test
    model.eval()
    for x,y in test_loader:
        test_pred = model(x)           
        test_loss = criterion(test_pred, y)
        if test_loss < best_mse:
            best_mse = test_loss
            best_weights = copy.deepcopy(model.state_dict())
            


print(best_mse)    

155.8818


In [327]:
from pathlib import Path

MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = 'demo_model.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(model.state_dict(), f=MODEL_SAVE_PATH)

In [328]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [329]:
predictions = []
actual = []

model.eval()
with torch.inference_mode():
    for x,y in test_loader:
        test_pred = model(x)
        predictions.append(test_pred)
        actual.append(y)
        test_loss = criterion(test_pred, y)


In [330]:
print(predictions[0:10])
print(actual[0:10])

[tensor([[120.4340, 159.0024, 137.2133],
        [120.8875, 159.6068, 137.7350]]), tensor([[130.3494, 172.1372, 148.5572],
        [126.8255, 167.4635, 144.5270]]), tensor([[114.8313, 151.5788, 130.7975],
        [ 97.9378, 129.1942, 111.4611]]), tensor([[119.5636, 157.8559, 136.2222],
        [123.7676, 163.4120, 141.0232]]), tensor([[121.3734, 160.2457, 138.2830],
        [105.7890, 139.5973, 120.4501]]), tensor([[ 88.7899, 117.0812, 100.9927],
        [120.8051, 159.4888, 137.6335]]), tensor([[ 92.1921, 121.5816, 104.8829],
        [130.1642, 171.8874, 148.3430]]), tensor([[112.9848, 149.1398, 128.6928],
        [120.5473, 159.1496, 137.3403]]), tensor([[130.3575, 172.1421, 148.5638],
        [119.6962, 158.0290, 136.3668]]), tensor([[130.4183, 172.2325, 148.6391],
        [100.9458, 133.1796, 114.9067]])]
[tensor([[ 31, 208, 249],
        [ 84, 179,  60]]), tensor([[113, 221, 120],
        [181,  61, 238]]), tensor([[239, 217, 198],
        [213, 129,  25]]), tensor([[ 84, 179,  60